In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [2]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-05-21&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [3]:
results.status_code

200

In [10]:
#Repeat for years 2013-2018
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "trade-fiction-paperback"
API_KEY = "keys.json"

bestsellers = []
weeks = []
num_calls_to_make = 1
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date(2018, 5, 21) - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s?list=%s&published-date=%s&api-key=%s" % (BOOKS_ROOT,LIST, curr_date, API_KEY)
        url = url.strip()
        print(url)
        r = requests.get(url)

        if r.status_code == 200:
            results = r.json()
            for result in results['results']:
                for book_details in result['book_details']:
                    book_info = {
                        'title': book_details['title'],
                        'description': book_details['description'],
                        'contributor': book_details['contributor'],
                        'author': book_details['author'],
                        'contributor_note': book_details['contributor_note'],
                        'price': book_details['price'],
                        'age_group': book_details['age_group'],
                        'publisher': book_details['publisher'],
                        'primary_isbn13': book_details['primary_isbn13'],
                        'primary_isbn10': book_details['primary_isbn10'],
                        'rank': result['rank'],
                        'rank_last_week': result['rank_last_week'],
                        'weeks_on_list': result['weeks_on_list'],
                        'asterisk': result['asterisk'],
                        'dagger': result['dagger']
                    }
                    bestsellers.append(book_info)
                    weeks.append(str(curr_date))
                
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)

2018-05-21
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-05-21&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-05-14
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-05-14&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-05-07
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-05-07&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-04-30
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-04-30&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-04-23
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-04-23&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-04-16
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-fiction-paperback&published-date=2018-04-16&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2018-04-09
https://api.nytimes.com/svc/books/v3/lists.json?list=trade-

In [11]:
trade_fiction_paperback2 = pd.json_normalize(bestsellers)

In [12]:
trade_fiction_paperback2.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,INTO THE WATER,In this psychological thriller by the author o...,by Paula Hawkins,Paula Hawkins,,0.00,,Riverhead,9780735211223,0735211221,1,1,2,0,0
1,THE SUN AND HER FLOWERS,A second collection of poetry from the author ...,by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449486792,1449486797,2,3,32,0,0
2,MILK AND HONEY,"A collection of poetry about love, loss, traum...",by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449474256,144947425X,3,5,109,0,0
3,THE HANDMAID'S TALE,"In the Republic of Gilead's dystopian future, ...",by Margaret Atwood,Margaret Atwood,,0.00,,Anchor,9780385490818,038549081X,4,2,63,1,0
4,COME SUNDOWN,A Montana family is tested when a long-missing...,by Nora Roberts,Nora Roberts,,0.00,,St. Martin's Griffin,9781250123084,1250123089,5,4,3,0,0


In [13]:
trade_fiction_paperback2.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
4185,CITY OF WOMEN,"In 1943 Berlin, a German soldier’s wife become...",by David R. Gillham,David R Gillham,,0.00,,Berkley,9780425252963,0425252965,16,0,2,0,0
4186,THE PARIS WIFE,Hadley Richardson comes into her own as Ernest...,by Paula McLain,Paula McLain,,0.00,,Ballantine,9780345521316,0345521315,17,0,26,0,0
4187,THE ART FORGER,A talented yet struggling Boston artist agrees...,by B. A. Shapiro,B A Shapiro,,0.00,,Algonquin,9781616203160,1616203161,18,0,1,0,0
4188,THE LANGUAGE OF FLOWERS,A young woman’s gift for flowers helps her cha...,by Vanessa Diffenbaugh,Vanessa Diffenbaugh,,0.00,,Ballantine,9780345525550,0345525558,19,0,49,0,0
4189,THE DOG STARS,After a flu pandemic kills much of the populat...,by Peter Heller,Peter Heller,,0.00,,Vintage Contemporaries,9780307950475,0307950476,20,0,1,0,0


In [14]:
len(trade_fiction_paperback2)

4190

In [15]:
len(weeks)

4190

In [16]:
trade_fiction_paperback2['week'] = weeks

In [18]:
trade_fiction_paperback2.head(5)


,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
0,INTO THE WATER,In this psychological thriller by the author o...,by Paula Hawkins,Paula Hawkins,,0.00,,Riverhead,9780735211223,0735211221,1,1,2,0,0,2018-05-21
1,THE SUN AND HER FLOWERS,A second collection of poetry from the author ...,by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449486792,1449486797,2,3,32,0,0,2018-05-21
2,MILK AND HONEY,"A collection of poetry about love, loss, traum...",by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449474256,144947425X,3,5,109,0,0,2018-05-21
3,THE HANDMAID'S TALE,"In the Republic of Gilead's dystopian future, ...",by Margaret Atwood,Margaret Atwood,,0.00,,Anchor,9780385490818,038549081X,4,2,63,1,0,2018-05-21
4,COME SUNDOWN,A Montana family is tested when a long-missing...,by Nora Roberts,Nora Roberts,,0.00,,St. Martin's Griffin,9781250123084,1250123089,5,4,3,0,0,2018-05-21


In [19]:
trade_fiction_paperback2['rank'] = trade_fiction_paperback2['rank'].astype(str)

In [20]:
filter_15= trade_fiction_paperback2['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [22]:
trade_fiction_paperback2 = trade_fiction_paperback2[filter_15]

In [24]:
trade_fiction_paperback2.head(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
0,INTO THE WATER,In this psychological thriller by the author o...,by Paula Hawkins,Paula Hawkins,,0.00,,Riverhead,9780735211223,0735211221,1,1,2,0,0,2018-05-21
1,THE SUN AND HER FLOWERS,A second collection of poetry from the author ...,by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449486792,1449486797,2,3,32,0,0,2018-05-21
2,MILK AND HONEY,"A collection of poetry about love, loss, traum...",by Rupi Kaur,Rupi Kaur,,0.00,,Andrews McMeel,9781449474256,144947425X,3,5,109,0,0,2018-05-21
3,THE HANDMAID'S TALE,"In the Republic of Gilead's dystopian future, ...",by Margaret Atwood,Margaret Atwood,,0.00,,Anchor,9780385490818,038549081X,4,2,63,1,0,2018-05-21
4,COME SUNDOWN,A Montana family is tested when a long-missing...,by Nora Roberts,Nora Roberts,,0.00,,St. Martin's Griffin,9781250123084,1250123089,5,4,3,0,0,2018-05-21
5,CRAZY RICH ASIANS,A New Yorker gets a surprise when she spends t...,by Kevin Kwan,Kevin Kwan,,0.00,,Anchor,9780345803788,0345803787,6,8,3,0,0,2018-05-21
6,"SING, UNBURIED, SING",A 13-year-old boy comes of age in Mississippi ...,by Jesmyn Ward,Jesmyn Ward,,0.00,,Scribner,9781501126079,1501126075,7,0,1,0,0,2018-05-21
7,TWO KINDS OF TRUTH,While Harry Bosch investigates the murder of t...,by Michael Connelly,Michael Connelly,,0.00,,Grand Central,9781455524174,1455524174,8,7,2,0,0,2018-05-21
8,READY PLAYER ONE,"It’s 2044, life on a resource-depleted Earth i...",by Ernest Cline,Ernest Cline,,0.00,,Broadway,9780307887443,0307887448,9,6,58,1,0,2018-05-21
9,SMALL GREAT THINGS,"A medical crisis entangles a black nurse, a wh...",by Jodi Picoult,Jodi Picoult,,0.00,,Ballantine,9780345544971,0345544978,10,9,12,0,0,2018-05-21


In [25]:
len(trade_fiction_paperback2)

3610

In [27]:
#and a column indicating which list these come from
k2 = 3610
weekly_fiction_list = ['Trade Paperback Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3610

In [29]:
#add nyt_list as a column
trade_fiction_paperback2['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
trade_fiction_paperback2.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
4180,THE ORPHAN MASTER’S SON,"In a series of picaresque adventures, a young ...",by Adam Johnson,Adam Johnson,,0.00,,Random House,9780812982626,0812982622,11,0,6,0,0,2013-06-03,Trade Paperback Fiction
4181,THE HARBINGER,A man tells of his encounters with a mysteriou...,by Jonathan Cahn,Jonathan Cahn,,0.00,,FrontLine,9781616386108,161638610X,12,0,73,0,0,2013-06-03,Trade Paperback Fiction
4182,A GAME OF THRONES,In the frozen wastes to the north of Winterfel...,by George R.R. Martin,George RR Martin,,0.00,,Bantam,9780553381689,0553381687,13,0,49,1,0,2013-06-03,Trade Paperback Fiction
4183,DARK PLACES,"A woman who, as a child, was spared when her m...",by Gillian Flynn,Gillian Flynn,,0.00,,Broadway,9780307341570,0307341577,14,0,25,0,0,2013-06-03,Trade Paperback Fiction
4184,BRING UP THE BODIES,"In this sequel to “Wolf Hall,” Thomas Cromwell...",by Hilary Mantel,Hilary Mantel,,0.00,,Picador,9781250024176,125002417X,15,0,3,1,0,2013-06-03,Trade Paperback Fiction


In [30]:
#save to csv
trade_fiction_paperback2.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2013to2018_trade_paperback_fiction.csv", index=False)